# Pythonic objects

- toc: true
- badges: true
- comments: true
- categories: [python]

Practicing the use of special methods.

## An artificially powerful French card deck

Based on part 4 in Fluent Python.

In [1]:
import collections
import functools
import numbers
import operator

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self, cards=None):
        if cards is None:
            self._cards = [Card(rank, suit) for suit in self.suits
                                            for rank in self.ranks]
        else:
            self._cards = cards
        
    def __len__(self):
        return len(self._cards)
    
    # basic implementation returns a list
    def __getitem__(self, position):
        return self._cards[position]
    
    # make slicing return a FrenchDeck rather than a list
    # this is what requires the conditional init statement
    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._cards[index])
        if isinstance(index, numbers.Integral):
            return self._cards[index]
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))
        
    def __str__(self):
        cards = [(card.suit, card.rank) for card in self._cards]
        return str(tuple(cards))
    
    def __repr__(self):
        cards = [(card.suit, card.rank) for card in self._cards]
        return 'FrenchDeck({})'.format(cards)
    
    shortcut_names = 'abc'

    # access first three elements with abc shortcut
    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = self.shortcut_names.find(name)
            if 0 <= pos < len(self._cards):
                return self._cards[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))
        
    # avoid attribute setting to avoid confusion with abc shortcuts
    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1:
            if name in self.shortcut_names:
                error = 'readonly attribute {attr_name!r}'
            elif name.islower():
                error = "can't set attributes 'a' to 'z' in {cls_name!r}"
            else:
                error = ''
            if error:
                msg = error.format(attr_name=name, cls_name=cls.__name__)
                raise AttributeError(msg)
        super().__setattr__(name, value)
        
    # basic comparison
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    # more efficient comparison for large deck
    def __eq__(self, other):
        return (len(self) == len(other) and 
                all(a == b for a, b in zip(self, other)))
    
    def __hash__(self):
        hashes = map(hash, self._cards)
        return functools.reduce(operator.xor, hashes)
    
    # totally artificial format attribute
    def __format__(self, fmt):
        cards = ((format(card.rank, fmt), card.suit) 
                 for card in deck._cards[:3])
        return '{}, {}, {}'.format(*cards)
        
    
           
deck = FrenchDeck()
# print(deck[3:5])
# print(deck.a)
# deck[2:3]
ddeck = FrenchDeck()
format(deck, '-^5')

"('--2--', 'spades'), ('--3--', 'spades'), ('--4--', 'spades')"

FrenchDeck as a subclass of an existing ABC, `collections.MutableSequence`.

In [7]:
import collections

Card = collections.namedtuple('Card', ['card', 'suit'])

class FrenchDeck2(collections.MutableSequence):
    cards = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spandes clubs diamonds hearts'.split()
    
    def __init__(self):
        self._cards = [Card(card, suit) for suit in self.suits
                                    for card in self.cards]
        
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]
    
    def __iter__(self):
        return (c for c in self._cards)
    
    def __delitem__(self, position):
        del self._cards[position]
        
    def __setitem__(self, key, value):
        self._cards[key] = value
        
    def insert(self, position, value):
        self._cards.insert(position, value)
        
a = FrenchDeck2()

a.insert(1, 'hello')
len(a)
a[1] = 'ha'
len(a)
a[:3]
list(reversed(a[:5]))

[Card(card='5', suit='spandes'),
 Card(card='4', suit='spandes'),
 Card(card='3', suit='spandes'),
 'ha',
 Card(card='2', suit='spandes')]

## Sources

- [Fluent Python](https://www.oreilly.com/library/view/fluent-python/9781491946237/)